In [1]:
!pip install vecstack
!pip install sklearn-genetic
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from genetic_selection import GeneticSelectionCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector
!pip install mlxtend

In [2]:
pd.set_option('display.max_columns',None)#displaying long list of columns
pd.set_option('display.max_rows', None)#displaying long list of rows
pd.set_option('display.width', 1000)#width of window

In [3]:
train_data=pd.read_csv('/content/sample_data/train.csv')
test_data=pd.read_csv('/content/sample_data/test.csv')

In [4]:
print(train_data.shape)
print(test_data.shape)


(188318, 132)
(125546, 131)


In [5]:
train_data.isnull().sum().sort_values(ascending=False)

,0
id,0
cat83,0
cat97,0
cat96,0
cat95,0
cat94,0
cat93,0
cat92,0
cat91,0
cat90,0


In [6]:
test_data.isnull().sum().sort_values(ascending=False)

,0
id,0
cat98,0
cat96,0
cat95,0
cat94,0
cat93,0
cat92,0
cat91,0
cat90,0
cat89,0


In [ ]:
#NOW IMPUTE MISSING VALUES FOR THE OTHER COLUMNS=========================
#IMPUTE (SUBSTITUTE) MEAN VALUES FOR NaN IN NUMERIC COLUMNS
numeric=train_data.select_dtypes(include=['int','float64']).columns
for num in numeric:
  train_data[num]=train_data[num].fillna(train_data[num].mean())

In [7]:
#Extracting the Target Column before doing missing value substitutions and one-hot encoding======
y_train = train_data["loss"]#make copy of target column
X_train = train_data.drop(["loss"], axis=1) #extracting training data without the target column

print(X_train.shape)
print(y_train.shape)

(188318, 131)
(188318,)


In [8]:
combined_data = pd.concat([X_train, test_data], keys=[0,1])
# Get the categorical columns
train_cat_cols = combined_data.select_dtypes(exclude=['float', 'int']).columns

# One-hot encode categorical columns
combined_data = pd.get_dummies(combined_data, columns=train_cat_cols, drop_first=False)

# Ensure binary format (1/0) instead of True/False
combined_data = combined_data.astype(int)

combined_data.head(10)

id  cont1  cont2  cont3  cont4  cont5  cont6  cont7  cont8  cont9  cont10  cont11  cont12  cont13  cont14  cat1_A  cat1_B  cat2_A  cat2_B  cat3_A  cat3_B  cat4_A  cat4_B  cat5_A  cat5_B  cat6_A  cat6_B  cat7_A  cat7_B  cat8_A  cat8_B  cat9_A  cat9_B  cat10_A  cat10_B  cat11_A  cat11_B  cat12_A  cat12_B  cat13_A  cat13_B  cat14_A  cat14_B  cat15_A  cat15_B  cat16_A  cat16_B  cat17_A  cat17_B  cat18_A  cat18_B  cat19_A  cat19_B  cat20_A  cat20_B  cat21_A  cat21_B  cat22_A  cat22_B  cat23_A  cat23_B  cat24_A  cat24_B  cat25_A  cat25_B  cat26_A  cat26_B  cat27_A  cat27_B  cat28_A  cat28_B  cat29_A  cat29_B  cat30_A  cat30_B  cat31_A  cat31_B  cat32_A  cat32_B  cat33_A  cat33_B  cat34_A  cat34_B  cat35_A  cat35_B  cat36_A  cat36_B  cat37_A  cat37_B  cat38_A  cat38_B  cat39_A  cat39_B  cat40_A  cat40_B  cat41_A  cat41_B  cat42_A  cat42_B  cat43_A  cat43_B  cat44_A  cat44_B  cat45_A  cat45_B  cat46_A  cat46_B  cat47_A  cat47_B  cat48_A  cat48_B  cat49_A  cat49_B  cat50_A  cat50_B  \
0 0   1      0      0      0      0      0      0      0      0      0       0       0       0       0       0       1       0       0       1       1       0       0       1       1       0       1       0       1       0       1       0       0       1        1        0        0        1        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        0        1        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0   
  1   2      0      0      0      0      0      0      0      0      0       0       0       0       0       0       1       0       0       1       1       0       1       0       1       0       1       0       1       0       1       0       0       1        0        1        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0   
  2   5      0      0      0      0      0      0      0      0      0       0       0       0       0       0       1       0       0       1       1       0       1       0       0       1       1       0       1       0       1       0       0       1        0        1        0        1        0        1        0        1        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        0        1        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0        1        0   
  3  10      0      0

In [9]:
#Separate Train data and test data
train_1 = combined_data.xs(0)
test_1 = combined_data.xs(1)
print(train_1.shape)
print(test_1.shape)

(188318, 1191)
(125546, 1191)


In [10]:
# Split the training data into two parts: one for feature selection and one for training
X_train_fs, X_train_model, y_train_fs, y_train_model = train_test_split(
    train_1, y_train, test_size=0.7, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(
    X_train_fs, y_train_fs, test_size=0.5, random_state=42
)

In [11]:
X_val, X_test, y_val, y_test = train_test_split(
    X_train_fs, y_train_fs, test_size=0.5, random_state=42
)

In [12]:
#Decision tree Regressor

decisiont = DecisionTreeRegressor(random_state=42)
decisiont.fit(X_train_fs, y_train_fs)

# Predict and evaluate on the validation set
y_val_pred = decisiont.predict(X_val)

# Calculate regression metrics
val_rmse = root_mean_squared_error(y_val, y_val_pred)  
val_r2 = r2_score(y_val, y_val_pred)  

print(f"RMSE on the Validation Set: {val_rmse:.6f}")
print(f"R² on the Validation Set: {val_r2:.6f}")

RMSE on the Validation Set: 0.000000
R² on the Validation Set: 1.000000


In [13]:
X_train_fs_sample = X_train_fs.sample(5000, random_state=42)
y_train_fs_sample = y_train_fs[X_train_fs_sample.index]

In [14]:
#Random forest regressor

randomforest = RandomForestRegressor(random_state=42, n_estimators=10, n_jobs=-1)
randomforest.fit(X_train_fs_sample, y_train_fs_sample)

# Predict and evaluate on the validation set
y_val_pred = randomforest.predict(X_val)

# Calculate regression metrics
val_rmse = root_mean_squared_error(y_val, y_val_pred)
val_r2 = r2_score(y_val, y_val_pred)  

print(f"RMSE on the Validation Set: {val_rmse:.6f}")
print(f"R² on the Validation Set: {val_r2:.6f}")

RMSE on the Validation Set: 2217.004961
R² on the Validation Set: 0.400960


In [15]:
# MLP Regressor

mlp_regressor = MLPRegressor(
    hidden_layer_sizes=50,
    activation='relu',
    learning_rate='adaptive',
    max_iter=300,
    )

mlp_regressor.fit(X_train_fs_sample, y_train_fs_sample)

# Make predictions on the validation set
y_val_pred = mlp_regressor.predict(X_val)

# Calculate regression metrics
val_rmse = root_mean_squared_error(y_val, y_val_pred)
val_r2 = r2_score(y_val, y_val_pred)  # R² score

print(f"RMSE on the Validation Set: {val_rmse:.6f}")
print(f"R² on the Validation Set: {val_r2:.6f}")


RMSE on the Validation Set: 2871.882650
R² on the Validation Set: -0.005207


In [16]:
from sklearn.ensemble import GradientBoostingRegressor
print("\nEnsemble Methods Predictions using gradient boosting\n")

# Features used by each model
model_features = {
    decisiont: X_train_fs_sample.columns.tolist(),
    randomforest: X_train_fs_sample.columns.tolist(),
    mlp_regressor: X_train_fs_sample.columns.tolist(),
}

# list of all unique features
all_features = list(set(sum(model_features.values(), [])))

# Ensuring all models and data use the same features
predictions = []
for model in [decisiont, randomforest, mlp_regressor]:
    # Subset data for the current model's features
    X_val_subset = X_val[model_features[model]]
    pred = model.predict(X_val_subset)
    predictions.append(pred)

# Subset training and validation data for stacking
X_train_stack = X_train_model[all_features]
X_val_stack = X_val[all_features]

#Stack the predictions using vecstack
S_Train, S_Test = stacking([decisiont, randomforest, mlp_regressor],
                            X_train_stack, y_train_model, X_val_stack,
                            regression=True,
                            mode='oof_pred_bag',
                            needs_proba=False,
                            save_dir=None,
                            metric=mean_squared_error,
                            n_folds=2,
                            stratified=False,
                            shuffle=True,
                            random_state=0,
                            verbose=2)

meta_learner = GradientBoostingRegressor()
meta_learner.fit(S_Train, y_train_model)
# Make final predictions using the meta-learner
final_predictions = meta_learner.predict(S_Test)


Ensemble Methods Predictions using gradient boosting

task:         [regression]
metric:       [mean_squared_error]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [DecisionTreeRegressor]
    fold  0:  [7910712.74506141]
    fold  1:  [7907699.59011710]
    ----
    MEAN:     [7909206.16758925] + [1506.57747215]
    FULL:     [7909206.17901804]

model  1:     [RandomForestRegressor]
    fold  0:  [4384044.78640045]
    fold  1:  [4709510.91073606]
    ----
    MEAN:     [4546777.84856826] + [162733.06216781]
    FULL:     [4546776.61408670]

model  2:     [MLPRegressor]
    fold  0:  [4358766.43576202]
    fold  1:  [5864699.70772998]
    ----
    MEAN:     [5111733.07174600] + [752966.63598398]
    FULL:     [5111727.35979410]



In [17]:
# Hyperparameter space for Gradient Boosting
param_dist = {
    'n_estimators': [50, 100],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5],
}

# Initialize the GradientBoostingClassifier
model = GradientBoostingRegressor()

# RandomizedSearchCV to tune hyperparameters
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=5, cv=3, random_state=42, verbose=2)

# Fit the RandomizedSearchCV on the training data (stacked features)
random_search.fit(S_Train, y_train_model)


print("Best hyperparameters for Gradient Boosting meta-model:", random_search.best_params_)

# Use the best estimator (meta-model) from the random search
best_model = random_search.best_estimator_

# Re-train the meta-model with the best hyperparameters
best_model.fit(S_Train, y_train_model)

# Predict using the best model on the test data
y_pred = best_model.predict(S_Test)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=100; total time=  18.9s
[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=100; total time=  17.3s
[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=100; total time=  16.5s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time=  25.3s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time=  18.1s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time=  16.6s
[CV] END ...learning_rate=0.05, max_depth=3, n_estimators=50; total time=   8.4s
[CV] END ...learning_rate=0.05, max_depth=3, n_estimators=50; total time=   8.0s
[CV] END ...learning_rate=0.05, max_depth=3, n_estimators=50; total time=   8.9s
[CV] END ...learning_rate=0.1, max_depth=5, n_estimators=100; total time=  27.5s
[CV] END ...learning_rate=0.1, max_depth=5, n_estimators=100; total time=  28.0s
[CV] END ...learning_rate=0.1, max_depth=5, n_est

In [19]:
test_1.shape

(125546, 1191)

In [ ]:
# Ensure test data uses the same features
X_test_1 = test_1[all_features]

S_Test_1 = []
for model in [decisiont, randomforest, mlp_regressor]:
  X_test_subset = X_test_1[model_features[model]]  # Subset test data for model's features
  pred = model.predict(X_test_subset)  # Predict using the base model
  S_Test_1.append(pred)

# Stack predictions horizontally for meta-model
S_Test_1 = np.column_stack(S_Test_1)

# Use the best meta-model to predict on the test set
final_test_predictions = best_model.predict(S_Test_1)

# Print the predictions
print("Final Test Predictions:", final_test_predictions)


In [ ]:
test_ids = test_data['id']
# Create a submission DataFrame
submission = pd.DataFrame({
    'id': test_ids,
    'loss': final_test_predictions
})

In [ ]:
# Save the submission file
submission.to_csv(index=False)

Submission file saved to: C:/Users/kriti/Downloads/submission.csv
